In [44]:
import pandas as pd
import pandas_profiling
import numpy as np
import matplotlib.pyplot as plt
from IPython.display import display, HTML
import plotly.express as px
import plotly.graph_objects as go
import seaborn as sns
import calendar
pd.set_option('display.max_colwidth', -1)
pd.set_option('display.max_columns', None)  
from wordcloud import WordCloud, STOPWORDS
import folium
from folium import plugins

# 1. Merge datasets

In [ ]:
data_location = '/Users/anamaria/Desktop/dev/security_project/datasets/NUSE 934 611(M) 2017-2018.dsv'
data2018=pd.read_csv(data_location,delimiter="|")

In [ ]:
data_location = '/Users/anamaria/Desktop/dev/security_project/datasets/NUSE 934-611-611M ENERO2019.csv'
data2019=pd.read_csv(data_location,delimiter=";")

In [ ]:
frames = [data2018, data2019]
data = pd.concat(frames)
merged_nuse = data.loc[data['TIPO_DETALLE'] == '934 - RIÑA']
merged_nuse.reset_index(inplace=True)
merged_nuse.drop(columns=['index'], inplace=True)

In [ ]:
merged_nuse

In [ ]:
merged_nuse.to_csv(r'/Users/anamaria/Desktop/dev/security_project/datasets/merged_nuse.csv',index=None)

# 2. Rebuild missing data

## Methods to rebuild

In [45]:
import import_ipynb
import ws_address
from selenium.common.exceptions import TimeoutException
import re

In [46]:
def find_between( s, first, last ):
    try:
        start = s.index( first ) + len( first )
        end = s.index( last, start )
        return s[start:end]
    except ValueError:
        return ""

In [47]:
tags = ["Dirección ingresada: ","Dirección encontrada: ","Tipo dirección: ","Código postal: ","Sector catastral: ",
        "UPZ: ","Localidad: ","Latitud: ","Longitud: ","CHIP: "]
def parse_address_ws(ws_result):
    location = {}
    for idx in range(len(tags)-1):
        location[tags[idx].replace(': ','')] = find_between(ws_result,tags[idx],tags[idx+1])
    return location

In [48]:
def assign_upz(original_df,index,UPZ_ws_field):
    original_df.at[index,'COD_UPZ'] = find_between(UPZ_ws_field, '(', ')')
    original_df.at[index,'UPZ'] = find_between(UPZ_ws_field, '', ' (')

In [49]:
def rebuild_location_in_nuse(original_df, index, driver):
    address = original_df.at[index,'STR_DIRECCION_INCIDENTE']
    print(address)
    result_ws = ws_address.web_scrap_address(driver,address)
    ws_address.delete_address(driver,address)
    print(result_ws)

    if result_ws != "Not found":
        parsed_result = parse_address_ws(result_ws)
        print(parsed_result)
        original_df.at[index,'LATITUD'] = float(parsed_result['Latitud'])
        original_df.at[index,'LONGITUD'] = float(parsed_result['Longitud'])
        original_df.at[index,'LOCALIDAD'] = parsed_result['Localidad']
        #original_df.at[index,'COD_LOCALIDAD'] = '-'
        original_df.at[index,'SEC_CATASTRAL'] = parsed_result['Sector catastral']
        assign_upz(original_df,index,parsed_result['UPZ'])
        return "Rebuilt"
    else:
        return "Not processed"


In [50]:
def rebuild_address_in_nuse(original_df, index):
    log_text = original_df.at[index,'LOG_TEXT']
    address_found = re.search(address_regex,log_text)

    if address_found != None:
        parsed_address = clean_address(address_found)
        print(parsed_address.strip())
        original_df.at[index,'STR_DIRECCION_INCIDENTE'] = parsed_address.strip()
        return "Rebuilt"
    else:
        original_df.at[index,'STR_DIRECCION_INCIDENTE'] = 'ND'
        return "Not processed"

def clean_address(address_found):
    exclude_char_list = ['~','/','*','(',')']
    one_occurrence = address_found.group().split(',,,')[0].replace(',',' ')
    final_address = one_occurrence
    
    for char in exclude_char_list:
        if char in one_occurrence:
            final_address = final_address.split(char)[0]
            
    numbers_in_substring = re.sub('[^0-9]','', final_address)
    numbers_proportion = len(numbers_in_substring)/len(final_address)
    
    if numbers_proportion < 0.2:
        final_address = 'ND'
    
    return final_address


## Implement rebuild methods

In [51]:
data_location = '/Users/anamaria/Desktop/dev/security_project/datasets/merged_nuse.csv'
merged_nuse=pd.read_csv(data_location,delimiter=",")


In [52]:
pd.DataFrame({"Tipo de dato":merged_nuse.dtypes.values,
              "Celdas con valor '-'":(merged_nuse == '-').sum().values,
              "Celdas con valor ''":(merged_nuse == '').sum().values,
              "Celdas con valor ' '":(merged_nuse == ' ').sum().values,
              "Celdas vacías": merged_nuse.isna().sum().values},
             index=merged_nuse.columns)

/Users/anamaria/Desktop/dev/security_project/security_venv/lib/python3.7/site-packages/pandas/core/ops/__init__.py:1115: FutureWarning:

elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison



,Tipo de dato,Celdas con valor '-',Celdas con valor '',Celdas con valor ' ',Celdas vacías
STR_NUMERO_INTERNO,object,0,0,0,0
FECHA,object,0,0,0,0
HORA,int64,0,0,0,0
ANIO,int64,0,0,0,0
MES,int64,0,0,0,0
COD_LOCALIDAD,int64,0,0,0,0
LOCALIDAD,object,0,0,0,0
COD_UPZ,object,2453,0,0,0
UPZ,object,2453,0,0,0
COD_SEC_CATAST,object,2453,0,0,0


In [53]:
df1 = merged_nuse.loc[merged_nuse['COD_UPZ'] == '-']
df2 = merged_nuse.loc[merged_nuse['UPZ'] == '-']
df3 = merged_nuse.loc[merged_nuse['COD_SEC_CATAST'] == '-']
df4 = merged_nuse.loc[merged_nuse['SEC_CATASTRAL'] == '-']
df5 = merged_nuse.loc[merged_nuse['COD_BARRIO'] == '-']
df6 = merged_nuse.loc[merged_nuse['BARRIO'] == '-']

In [ ]:
df1.equals(df2) and df1.equals(df3) and df1.equals(df4) and df1.equals(df5) and df1.equals(df6)

### Rebuild address through log_text

In [54]:
#Try to rebuild missing address through log_text field
df_empty_locations_without_address = merged_nuse.loc[merged_nuse['STR_DIRECCION_INCIDENTE'] == '-']
list_idx_rebuild_address = list(df_empty_locations_without_address.index.values)

In [55]:
len(list_idx_rebuild_address)

36305

In [56]:
address_regex= '(CL|DG|KR|TV)+\s\d+.*(,,)'
registers_to_process = len(list_idx_rebuild_address)
rebuilt_registers = 0
registers_not_processed = 0
other_condition_counter = 0

for index in list_idx_rebuild_address:
    state = rebuild_address_in_nuse(rebuild_nuse1, index)
    
    if state == "Rebuilt":
        rebuilt_registers += 1
    elif state == "Not processed":
        registers_not_processed += 1
    else:
        other_condition_counter += 1
    
    print('Rebuilt registers: ',rebuilt_registers,'/',registers_to_process)
    print('Registers not processed: ',registers_not_processed, '/', registers_to_process)

Rebuilt registers:  0 / 36305
Registers not processed:  1 / 36305
Rebuilt registers:  0 / 36305
Registers not processed:  2 / 36305
Rebuilt registers:  0 / 36305
Registers not processed:  3 / 36305
Rebuilt registers:  0 / 36305
Registers not processed:  4 / 36305
Rebuilt registers:  0 / 36305
Registers not processed:  5 / 36305
Rebuilt registers:  0 / 36305
Registers not processed:  6 / 36305
CL 63D SUR 88A 06
Rebuilt registers:  1 / 36305
Registers not processed:  6 / 36305
CL 26 SUR 82 53
Rebuilt registers:  2 / 36305
Registers not processed:  6 / 36305
CL 151 13A 50 IN 3 AP 14 03
Rebuilt registers:  3 / 36305
Registers not processed:  6 / 36305
Rebuilt registers:  3 / 36305
Registers not processed:  7 / 36305
CL 97 SUR 3B 24
Rebuilt registers:  4 / 36305
Registers not processed:  7 / 36305
KR 30 72
Rebuilt registers:  5 / 36305
Registers not processed:  7 / 36305
KR 65 98  PI 2
Rebuilt registers:  6 / 36305
Registers not processed:  7 / 36305
KR 94 BIS 6A
Rebuilt registers:  7 / 363

Rebuilt registers:  10 / 36305
Registers not processed:  125 / 36305
Rebuilt registers:  10 / 36305
Registers not processed:  126 / 36305
Rebuilt registers:  10 / 36305
Registers not processed:  127 / 36305
Rebuilt registers:  10 / 36305
Registers not processed:  128 / 36305
Rebuilt registers:  10 / 36305
Registers not processed:  129 / 36305
Rebuilt registers:  10 / 36305
Registers not processed:  130 / 36305
Rebuilt registers:  10 / 36305
Registers not processed:  131 / 36305
Rebuilt registers:  10 / 36305
Registers not processed:  132 / 36305
Rebuilt registers:  10 / 36305
Registers not processed:  133 / 36305
Rebuilt registers:  10 / 36305
Registers not processed:  134 / 36305
Rebuilt registers:  10 / 36305
Registers not processed:  135 / 36305
Rebuilt registers:  10 / 36305
Registers not processed:  136 / 36305
Rebuilt registers:  10 / 36305
Registers not processed:  137 / 36305
Rebuilt registers:  10 / 36305
Registers not processed:  138 / 36305
Rebuilt registers:  10 / 36305
Reg

Registers not processed:  271 / 36305
Rebuilt registers:  12 / 36305
Registers not processed:  272 / 36305
Rebuilt registers:  12 / 36305
Registers not processed:  273 / 36305
Rebuilt registers:  12 / 36305
Registers not processed:  274 / 36305
Rebuilt registers:  12 / 36305
Registers not processed:  275 / 36305
Rebuilt registers:  12 / 36305
Registers not processed:  276 / 36305
Rebuilt registers:  12 / 36305
Registers not processed:  277 / 36305
Rebuilt registers:  12 / 36305
Registers not processed:  278 / 36305
Rebuilt registers:  12 / 36305
Registers not processed:  279 / 36305
Rebuilt registers:  12 / 36305
Registers not processed:  280 / 36305
Rebuilt registers:  12 / 36305
Registers not processed:  281 / 36305
Rebuilt registers:  12 / 36305
Registers not processed:  282 / 36305
Rebuilt registers:  12 / 36305
Registers not processed:  283 / 36305
Rebuilt registers:  12 / 36305
Registers not processed:  284 / 36305
Rebuilt registers:  12 / 36305
Registers not processed:  285 / 36

Rebuilt registers:  21 / 36305
Registers not processed:  470 / 36305
Rebuilt registers:  21 / 36305
Registers not processed:  471 / 36305
Rebuilt registers:  21 / 36305
Registers not processed:  472 / 36305
Rebuilt registers:  21 / 36305
Registers not processed:  473 / 36305
Rebuilt registers:  21 / 36305
Registers not processed:  474 / 36305
Rebuilt registers:  21 / 36305
Registers not processed:  475 / 36305
Rebuilt registers:  21 / 36305
Registers not processed:  476 / 36305
Rebuilt registers:  21 / 36305
Registers not processed:  477 / 36305
Rebuilt registers:  21 / 36305
Registers not processed:  478 / 36305
Rebuilt registers:  21 / 36305
Registers not processed:  479 / 36305
Rebuilt registers:  21 / 36305
Registers not processed:  480 / 36305
Rebuilt registers:  21 / 36305
Registers not processed:  481 / 36305
Rebuilt registers:  21 / 36305
Registers not processed:  482 / 36305
Rebuilt registers:  21 / 36305
Registers not processed:  483 / 36305
Rebuilt registers:  21 / 36305
Reg

Rebuilt registers:  22 / 36305
Registers not processed:  594 / 36305
Rebuilt registers:  22 / 36305
Registers not processed:  595 / 36305
Rebuilt registers:  22 / 36305
Registers not processed:  596 / 36305
Rebuilt registers:  22 / 36305
Registers not processed:  597 / 36305
Rebuilt registers:  22 / 36305
Registers not processed:  598 / 36305
Rebuilt registers:  22 / 36305
Registers not processed:  599 / 36305
Rebuilt registers:  22 / 36305
Registers not processed:  600 / 36305
Rebuilt registers:  22 / 36305
Registers not processed:  601 / 36305
Rebuilt registers:  22 / 36305
Registers not processed:  602 / 36305
Rebuilt registers:  22 / 36305
Registers not processed:  603 / 36305
Rebuilt registers:  22 / 36305
Registers not processed:  604 / 36305
Rebuilt registers:  22 / 36305
Registers not processed:  605 / 36305
Rebuilt registers:  22 / 36305
Registers not processed:  606 / 36305
Rebuilt registers:  22 / 36305
Registers not processed:  607 / 36305
Rebuilt registers:  22 / 36305
Reg

...

Rebuilt registers:  773 / 36305
Registers not processed:  34873 / 36305
Rebuilt registers:  773 / 36305
Registers not processed:  34874 / 36305
Rebuilt registers:  773 / 36305
Registers not processed:  34875 / 36305
Rebuilt registers:  773 / 36305
Registers not processed:  34876 / 36305
Rebuilt registers:  773 / 36305
Registers not processed:  34877 / 36305
Rebuilt registers:  773 / 36305
Registers not processed:  34878 / 36305
Rebuilt registers:  773 / 36305
Registers not processed:  34879 / 36305
Rebuilt registers:  773 / 36305
Registers not processed:  34880 / 36305
Rebuilt registers:  773 / 36305
Registers not processed:  34881 / 36305
Rebuilt registers:  773 / 36305
Registers not processed:  34882 / 36305
Rebuilt registers:  773 / 36305
Registers not processed:  34883 / 36305
Rebuilt registers:  773 / 36305
Registers not processed:  34884 / 36305
Rebuilt registers:  773 / 36305
Registers not processed:  34885 / 36305
Rebuilt registers:  773 / 36305
Registers not processed:  34886 

Rebuilt registers:  774 / 36305
Registers not processed:  35000 / 36305
Rebuilt registers:  774 / 36305
Registers not processed:  35001 / 36305
Rebuilt registers:  774 / 36305
Registers not processed:  35002 / 36305
Rebuilt registers:  774 / 36305
Registers not processed:  35003 / 36305
Rebuilt registers:  774 / 36305
Registers not processed:  35004 / 36305
Rebuilt registers:  774 / 36305
Registers not processed:  35005 / 36305
Rebuilt registers:  774 / 36305
Registers not processed:  35006 / 36305
Rebuilt registers:  774 / 36305
Registers not processed:  35007 / 36305
Rebuilt registers:  774 / 36305
Registers not processed:  35008 / 36305
Rebuilt registers:  774 / 36305
Registers not processed:  35009 / 36305
Rebuilt registers:  774 / 36305
Registers not processed:  35010 / 36305
Rebuilt registers:  774 / 36305
Registers not processed:  35011 / 36305
Rebuilt registers:  774 / 36305
Registers not processed:  35012 / 36305
Rebuilt registers:  774 / 36305
Registers not processed:  35013 

Registers not processed:  35167 / 36305
Rebuilt registers:  778 / 36305
Registers not processed:  35168 / 36305
Rebuilt registers:  778 / 36305
Registers not processed:  35169 / 36305
Rebuilt registers:  778 / 36305
Registers not processed:  35170 / 36305
Rebuilt registers:  778 / 36305
Registers not processed:  35171 / 36305
Rebuilt registers:  778 / 36305
Registers not processed:  35172 / 36305
Rebuilt registers:  778 / 36305
Registers not processed:  35173 / 36305
Rebuilt registers:  778 / 36305
Registers not processed:  35174 / 36305
Rebuilt registers:  778 / 36305
Registers not processed:  35175 / 36305
Rebuilt registers:  778 / 36305
Registers not processed:  35176 / 36305
Rebuilt registers:  778 / 36305
Registers not processed:  35177 / 36305
Rebuilt registers:  778 / 36305
Registers not processed:  35178 / 36305
Rebuilt registers:  778 / 36305
Registers not processed:  35179 / 36305
KR 105F 72
Rebuilt registers:  779 / 36305
Registers not processed:  35179 / 36305
KR 105F 75
Re

Registers not processed:  35332 / 36305
Rebuilt registers:  790 / 36305
Registers not processed:  35333 / 36305
Rebuilt registers:  790 / 36305
Registers not processed:  35334 / 36305
Rebuilt registers:  790 / 36305
Registers not processed:  35335 / 36305
Rebuilt registers:  790 / 36305
Registers not processed:  35336 / 36305
Rebuilt registers:  790 / 36305
Registers not processed:  35337 / 36305
Rebuilt registers:  790 / 36305
Registers not processed:  35338 / 36305
Rebuilt registers:  790 / 36305
Registers not processed:  35339 / 36305
Rebuilt registers:  790 / 36305
Registers not processed:  35340 / 36305
Rebuilt registers:  790 / 36305
Registers not processed:  35341 / 36305
Rebuilt registers:  790 / 36305
Registers not processed:  35342 / 36305
KR 101B 156 21 PI 2
Rebuilt registers:  791 / 36305
Registers not processed:  35342 / 36305
Rebuilt registers:  791 / 36305
Registers not processed:  35343 / 36305
Rebuilt registers:  791 / 36305
Registers not processed:  35344 / 36305
Rebu

Rebuilt registers:  795 / 36305
Registers not processed:  35499 / 36305
Rebuilt registers:  795 / 36305
Registers not processed:  35500 / 36305
Rebuilt registers:  795 / 36305
Registers not processed:  35501 / 36305
Rebuilt registers:  795 / 36305
Registers not processed:  35502 / 36305
Rebuilt registers:  795 / 36305
Registers not processed:  35503 / 36305
Rebuilt registers:  795 / 36305
Registers not processed:  35504 / 36305
Rebuilt registers:  795 / 36305
Registers not processed:  35505 / 36305
Rebuilt registers:  795 / 36305
Registers not processed:  35506 / 36305
Rebuilt registers:  795 / 36305
Registers not processed:  35507 / 36305
Rebuilt registers:  795 / 36305
Registers not processed:  35508 / 36305
Rebuilt registers:  795 / 36305
Registers not processed:  35509 / 36305
Rebuilt registers:  795 / 36305
Registers not processed:  35510 / 36305


In [57]:
merged_nuse.to_csv(r'/Users/anamaria/Desktop/dev/security_project/datasets/rebuild_address_nuse_18112019.csv',index=None)

In [ ]:
pd.DataFrame({"Tipo de dato":merged_nuse.dtypes.values,
              "Celdas con valor '-'":(merged_nuse == '-').sum().values,
              "Celdas con valor ''":(merged_nuse == '').sum().values,
              "Celdas con valor ' '":(merged_nuse == ' ').sum().values,
              "Celdas vacías": merged_nuse.isna().sum().values},
             index=merged_nuse.columns)

### Rebuild location through address

In [ ]:
#Try to rebuild 'sector catastral', 'UPZ', 'localidad', 'latitud', 'longitud' through address
df_empty_locations_with_address = df1.loc[df1['STR_DIRECCION_INCIDENTE'] != '-']
list_idx_rebuild_location = list(df_empty_locations_with_address.index.values)

In [ ]:
#Rebuild 'sector catastral', 'UPZ', 'localidad', 'latitud', 'longitud' using web scraping
url='https://mapas.bogota.gov.co'
driver = ws_address.web_scrap_page(url)
registers_to_process = len(list_idx_rebuild)
rebuilt_registers = 0
registers_not_processed = 0
other_condition_counter = 0

for index in list_idx_rebuild:
    state = rebuild_location_in_nuse(merged_nuse, index, driver)
    
    if state == "Rebuilt":
        rebuilt_registers += 1
    elif state == "Not processed":
        registers_not_processed += 1
    else:
        other_condition_counter += 1
    
    print('Rebuilt registers: ',rebuilt_registers,'/',registers_to_process)
    print('Registers not processed: ',registers_not_processed, '/', registers_to_process)

In [ ]:
print(rebuilt_registers)
print(registers_not_processed)
print(other_condition_counter)

In [ ]:
merged_nuse.to_csv(r'/Users/anamaria/Desktop/dev/security_project/datasets/rebuild_nuse_13112019.csv',index=None)